In [14]:
import os

In [15]:
os.chdir("../")

In [16]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path
    transformer_path:Path

In [17]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories

In [18]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            transformer_path=config.transformer_path,
        )

        return data_transformation_config

In [19]:
import os
from mlProject import logger
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler,OneHotEncoder

In [20]:
class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config

    # I have done all advanced pre processing techniques in data validation step and validated data for next step
    # here I am going to split the data and finally transform using columntransformer


    def train_test_spliting(self):
        data = pd.read_csv(self.config.data_path)

        # Split the data into training and test sets.
        X = data.drop('price',axis=1)
        y = data['price']
        x_train,x_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)
        logger.info("Splited data into training and test sets")
        # logger.info(x_train.shape)
        # logger.info(x_test.shape)
        return x_train,x_test,y_train,y_test
    
    def transformation(self,x_train,x_test,y_train,y_test):
        
        cat_cols = ['brand', 'OS', 'gpu_type', 'processor_brand', 'processor_version']
        cat_transformer = Pipeline(
            steps=[
                ("encoder", OneHotEncoder(sparse=False,drop='first',handle_unknown='ignore')),
                ("scaler", StandardScaler()),
                ]
                )
        num_cols = [ 'spec_rating', 'Ram', 'ROM', 'ROM_type', 'display_size',
                    'resolution_width', 'resolution_height', 'warranty', 'cpu_core',
                    'cpu_threads', 'processor_gen']
        num_cat_transformer = Pipeline(
            steps=[
                # ("encoder", LabelEncoder()),
                ("scaler", StandardScaler()),
            ]
        )

        transformer = ColumnTransformer(
            transformers=[
                ('categorical_transformer',cat_transformer,cat_cols),
                ("numerical_transformer",num_cat_transformer,num_cols)
                ]
                )
        final_x_train = transformer.fit_transform(x_train)
        logger.info(f'training data transformed, shape:{final_x_train.shape}')
        final_x_test = transformer.transform(x_test)
        logger.info(f'testing data transformed, shape:{final_x_test.shape}')

        import joblib
        joblib.dump(transformer, self.config.transformer_path)
        logger.info("Transformer dumped at given location")


        return final_x_train,y_train,final_x_test,y_test
        

In [21]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    x_train,x_test,y_train,y_test = data_transformation.train_test_spliting()
    data_transformation.transformation(x_train,x_test,y_train,y_test)
except Exception as e:
    raise e

[2023-12-10 19:49:15,874: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-12-10 19:49:15,877: INFO: common: yaml file: params.yaml loaded successfully]
[2023-12-10 19:49:15,882: INFO: common: yaml file: schema.yaml loaded successfully]
[2023-12-10 19:49:15,884: INFO: common: created directory at: artifacts]
[2023-12-10 19:49:15,887: INFO: common: created directory at: artifacts/data_transformation]
[2023-12-10 19:49:15,921: INFO: 4940772: Splited data into training and test sets]
[2023-12-10 19:49:15,939: INFO: 4940772: training data transformed, shape:(714, 74)]
[2023-12-10 19:49:15,948: INFO: 4940772: testing data transformed, shape:(179, 74)]
[2023-12-10 19:49:15,953: INFO: 4940772: Transformer dumped at given location]


c:\Users\ten\Desktop\Laptop_Price\venv\lib\site-packages\sklearn\preprocessing\_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
c:\Users\ten\Desktop\Laptop_Price\venv\lib\site-packages\sklearn\preprocessing\_encoders.py:228: UserWarning: Found unknown categories in columns [0, 1, 4] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
